- P(w|wi-n+1) = Ct+lambda/N+B*lambda
on Ct es el numero d'aparicions del enegrama, N 

Només es cert en cas de que aquest caracter depengui dels n-1 anteriors. Quant mes gran sigui la sequencia, més real es aixó. El problema si el enegrama es massa gran el Ct serà cada cop més petit.  Si és més petit, és menys precís.

Fem servir smoothing pq hi ha enegramas que no estan al nostre corpus. 

- Posar doble espai al final i al inici del document per poder predir quin es el enegrama que indica l'inici/final (--a/a--)

- Altra tècnica de smoothing: Interpolació P(wi wi-1 wi-2) = lambda0 * P(wi, wi-1, wi-2) + lambda1 * P(wi, wi-1) + lambda2 * P(wi) on lambda0 + lambda1 + lambda2 = 1

El pes de cada lambda depén de quina es la millor probabilitat donat el corpus. La P basada en enegrames més llargs és més precisa pero si el corpus es mes petit, anira malament...

- Altra: Discount

Pabs(X =x) = (C(X=x)-delta)/N if C(w1...wn) > 0
            ((B-No)*delta/No)/N otherwise

- Altra: Backing-off

Si trobem el enegrama fem servir la seva prob. Si no, la del n-1 grama. Si no, la del n-2 grama...
S'han d'aproximar una certa alfa i una certa belta per aasegurar que el sumatori de les probabilitats siguin 1.
Es poden donar valors aporximats i ja està, normalment es farien amb good tuning.

- Probar con diferente numero de enegramas de cara al análisis



**Càrrega de dades**

In [ ]:
import spacy

from pathlib import Path


In [5]:
path = Path(r"C:\Users\Aya\UPC\PLH\Projecte 1\langId\eng_trn.txt")
text = path.read_text(encoding="utf-8")


**Preprocés**

***Eliminar els dígits***

In [9]:
import re

text_clean = re.sub(r"\d+", "", text)  # eliminar números

text = text_clean

***Convertir el text a minúscula***

In [10]:
text = text.lower()  # convertir a minúsculas

***Substituir espais en blanc per un sol***

In [11]:
text = re.sub(r"\s+", " ", text).strip()  # eliminar espacios extra

**Eliminar caràcters extranys**

In [ ]:
weird_chars = ["—", "…", ':', "“", "”", "‘", "’", "•", "–", "—", "―", "«", "»", '%', '&', '$', '@', '(', ')', '[', ']', '{', '}', '|', '\\', '/', '<', '>', '^', '~', '`']
for char in weird_chars:
    text = text.replace(char, " ")

***Concatenar totes les frases amb un espai doble al mig***

In [13]:
sentences = [line.strip() for line in text.splitlines() if line.strip()]
text_joined = "  ".join(sentences)
text = text_joined
print(text[:500])  # vistazo rápido

. .  how the mighty have fallen. . .  major companies coming out with their latest quarterly numbers include dave   buster's, men's wearhouse, palo alto networks, box, barnes   noble, pep boys-manny, moe   jack, ollie's bargain outlet, davidstea, lululemon athletica and kroger. january , magazine why are there so many magna cartas? : now watching up next word association with brad woodhouse the president of the democratic american bridge pac on the gop presidential field. : autoplay autoplay cop


DEJAR DE MOMENTO

In [ ]:
import re, math
from pathlib import Path
from collections import Counter

# ─── Función de preprocesado completa ───────────────────────────────────

weird_chars = [
    "—", "…", ":", "\u201c", "\u201d", "\u2018", "\u2019", "•", "–",
    "―", "«", "»", "%", "&", "$", "@", "(", ")", "[", "]", "{", "}",
    "|", "\\", "/", "<", ">", "^", "~", "`", "#", "=", "+", "*", "_",
]

def preprocess(raw_text):
    """Aplica todos los pasos de preprocesado al texto crudo."""
    processed = []
    for line in raw_text.splitlines():
        line = re.sub(r"\d+", "", line)          # eliminar dígitos
        line = line.lower()                       # minúsculas
        for ch in weird_chars:
            line = line.replace(ch, " ")
        line = re.sub(r"\s+", " ", line).strip()  # colapsar espacios
        if line:
            processed.append(line)
    # Concatenar frases con doble espacio (marca de frontera)
    return "  ".join(processed)

# ─── Cargar y preprocesar todos los idiomas ─────────────────────────────

languages = ["deu", "eng", "fra", "ita", "nld", "spa"]
base_path = Path("langId")

train_texts = {}
test_texts  = {}

for lang in languages:
    train_texts[lang] = preprocess((base_path / f"{lang}_trn.txt").read_text(encoding="utf-8"))
    test_texts[lang]  = preprocess((base_path / f"{lang}_tst.txt").read_text(encoding="utf-8"))

print("Idiomas cargados:")
for lang in languages:
    print(f"  {lang}  →  train {len(train_texts[lang]):>8,} chars | test {len(test_texts[lang]):>7,} chars")

**Divisió en trigrames**

**Model d'enegrames**

In [ ]:
enegrames = []

In [ ]:
vocabulari = set(text)
B = len(vocabulari)

lamda = 0.5

def probabilitat_linstone(enegrama, lamda):
    count_enegrama = enegrames.count(enegrama)
    minigrama = enegrama[:-1]
    count_minigrama = enegrames.count(minigrama)

    return ((count_enegrama+lamda)/(count_minigrama+B*lamda))